In [148]:
import hashlib
import base58
import hashlib
import requests
import csv
import os
import warnings
import gc
import argparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import ast
import re
import time
import csv
import hashlib
import os
import psutil
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import hashlib
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import timedelta
from datetime import datetime
import json

# Suppress all warnings
warnings.filterwarnings("ignore")

import logging

# Set up logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)  # Set the logging level

# Remove any default handlers associated with the logger
logger.handlers = []

# Create a file handler
handler = logging.FileHandler('worklog.log', encoding='utf-8')
# Create a rotating file handler
handler.setLevel(logging.DEBUG)  # Set the handler level

# Create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(handler)

# Disable propagation to avoid printing to the console
logger.propagate = False

In [95]:
import requests
import datetime
TRON_NODE_URL = "https://trx-rpc.transatron.io"


# Кеш для адрес
address_cache = {}

def hex_to_base58(hex_address):
    if hex_address in address_cache:
        return address_cache[hex_address]
    if not hex_address.startswith("41"):
        raise ValueError("Invalid Tron address: Must start with '41'")

    address_bytes = bytes.fromhex(hex_address)
    checksum = hashlib.sha256(hashlib.sha256(address_bytes).digest()).digest()[:4]
    address_with_checksum = address_bytes + checksum
    base58_address = base58.b58encode(address_with_checksum).decode()
    address_cache[hex_address] = base58_address
    return base58_address


def hex_to_tron_address(hex_address):
    """
    Convert a 20-byte hex address to a TRON address with the '41' prefix.
    """
    return "41" + hex_address[-40:]

def get_block_transactions(block_number):
    """
    Fetch and process transactions for a specific block using the TRON Full Node.
    """
    url = f"{TRON_NODE_URL}/wallet/gettransactioninfobyblocknum"
    
    # Send request to get transaction info for the block
    response = requests.post(url, json={"num": block_number})
    response.raise_for_status()
    
    # Parse the response JSON
    block_data = response.json()

    # Add block timestamp to each transaction
    for tx in block_data:
        # Attach block timestamp to each transaction
        tx["block_timestamp"] = block_data[0]["blockTimeStamp"]  # Assuming all transactions share the same timestamp in the block
        tx["block_number"] = block_number

    return block_data

In [12]:
def filter_successful_transactions(transactions):
    return [
        tx for tx in transactions
        if tx.get("receipt", {}).get("result") == "SUCCESS"
    ]

In [44]:
transactions=get_block_transactions(57833080)
successful_transactions=filter_successful_transactions(transactions)

In [41]:
#successful_transactions

In [16]:
# Event Signatures
ADDED_BLACKLIST_TOPIC = "42e160154868087d6bfdc0ca23d96a1c1cfa32f1b72ba9ba27b69b98a0d819dc"
REMOVED_BLACKLIST_TOPIC = "d7e9ec6e6ecd65492dce6bf513cd6867560d49544421d0783ddf06e76c24470c"

def filter_blacklist_transactions(transactions):
    return [
        tx for tx in transactions
        if any(topic == ADDED_BLACKLIST_TOPIC or topic == REMOVED_BLACKLIST_TOPIC for topic in (tx.get("log", [{}])[0].get("topics", [])))
    ]

In [119]:
# Event Signatures
ADDED_BLACKLIST_TOPIC = "42e160154868087d6bfdc0ca23d96a1c1cfa32f1b72ba9ba27b69b98a0d819dc"
REMOVED_BLACKLIST_TOPIC = "d7e9ec6e6ecd65492dce6bf513cd6867560d49544421d0783ddf06e76c24470c"

class BlockTransactionCache:
    
    def __init__(self):
        self.cache = {}

    def get_filtered_transactions(self, block_number):
        """
        Returns filtered transactions for a block from memory cache if available.
        If not, fetches the transactions, filters for success, and stores them in the cache.
        """
        if block_number in self.cache:
            return self.cache[block_number]
        
        # Otherwise, fetch the block data and filter the transactions
        filtered_transactions = self.filter_block_transactions(block_number)
        
        # Store the filtered transactions in memory cache
        self.cache[block_number] = filtered_transactions
        
        return filtered_transactions

    def filter_block_transactions(self, block_number):
        """
        Fetches block transactions, filters them for success, 
        and then filters for Added/RemovedBlackList transactions.
        """
        block_data = self.fetch_block_data(block_number)
        successful_transactions = self.filter_successful_transactions(block_data)
        filtered_transactions = self.filter_added_removed_blacklist(successful_transactions)
        return filtered_transactions

    def fetch_block_data(self, block_number):
        """
        Fetch block data using your desired method (this is a placeholder).
        """
        url = f"{TRON_NODE_URL}/wallet/gettransactioninfobyblocknum"
        response = requests.post(url, json={"num": block_number})
        response.raise_for_status()
        return response.json()

    def filter_successful_transactions(self, transactions):
        """
        Filter the transactions to include only successful ones (receipt.result == 'SUCCESS').
        """
        return [
            tx for tx in transactions
            if tx.get("receipt", {}).get("result") == "SUCCESS"
        ]

    def filter_added_removed_blacklist(self, transactions):
        """
        Filter the transactions to include only those related to
        AddedBlackList and RemovedBlackList transactions.
        """
        filtered = []
        
        # Loop through each transaction in the block data
        for tx in transactions:
            logs = tx.get('log', [])
            
            for log in logs:
                if 'topics' in log:
                    topics = log['topics']
                    
                    if len(topics) > 0 and (topics[0] == ADDED_BLACKLIST_TOPIC or topics[0] == REMOVED_BLACKLIST_TOPIC):
                        filtered.append(tx)
                        break  # Once we find a match, no need to check other logs for this transaction
        
        return filtered

    def print_cached_transactions(self):
        """
        Print all cached transactions in memory.
        """
        if not self.cache:
            print("No transactions in cache.")
        else:
            for block_number, transactions in self.cache.items():
                print(f"Block Number: {block_number}")
                print("Transactions:")
                for tx in transactions:
                    print(tx)  # You can adjust the output format as needed
                print("-" * 80)
                
    def aggregate_cache_to_csv(self, filename="aggregated_transactions.csv"):
        """
        Aggregate all transactions in cache and write them to a CSV file.
        This flattens the nested JSON structure into a CSV format.
        """
        all_transactions = []
        
        # Iterate over all transactions in the cache
        for block_number, transactions in self.cache.items():
            for tx in transactions:
                flattened_tx = self.flatten_transaction(tx, block_number)
                all_transactions.append(flattened_tx)

        # Write the aggregated data to a CSV file
        if all_transactions:
            with open(filename, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=all_transactions[0].keys())
                writer.writeheader()
                writer.writerows(all_transactions)
            print(f"CSV file '{filename}' has been created successfully.")
        else:
            print("No transactions to write to CSV.")

    def flatten_transaction(self, tx, block_number):
        """
        Flatten the nested transaction structure into a flat dictionary that can be written to CSV.
        This method has been modified to match the functionality in the `process_logs` function.
        """
        flattened = {
            "transaction_id": tx.get("id"),  # tx_id
            "contract_address": tx.get("contract_address"),
            "block_number": block_number,
            "block_timestamp": datetime.utcfromtimestamp(tx.get("blockTimeStamp") / 1000) if tx.get("blockTimeStamp") else None,
            #"contract_result": tx.get("contractResult"),
            "receipt_result": tx.get("receipt", {}).get("result"),
            #"energy_usage": tx.get("receipt", {}).get("energy_usage"),
            #"net_usage": tx.get("receipt", {}).get("net_usage"),
        }
        
        # Process logs to determine event types
        logs = tx.get("log", [])
        for i, log in enumerate(logs):
            event_type = None
            tron_address = None

            # Check the event type based on topic_0
            topic_0 = log["topics"][0] if log.get("topics") else None
            if topic_0 == ADDED_BLACKLIST_TOPIC:
                event_type = "AddedBlackList"
            elif topic_0 == REMOVED_BLACKLIST_TOPIC:
                event_type = "RemovedBlackList"

            # Only process logs for relevant events
            if event_type:
                if len(log.get("topics", [])) > 1:
                    topic_1 = log["topics"][1]
                    tron_address = hex_to_base58(hex_to_tron_address(topic_1))

                # Adding processed logs details to flattened dict
                flattened["event_type"] = event_type
                flattened["tron_address"] = tron_address
                #flattened[f"log_{i}_contract_address"] = log.get("address")
                #flattened[f"log_{i}_block_timestamp"] = flattened.get("block_timestamp")

        return flattened

    
    def clear_cache(self):
            """
            Clears all data in the cache.
            """
            self.cache.clear()
            print("Cache has been cleared.")


In [120]:
# Create a cache instance
cache = BlockTransactionCache()

# Retrieve filtered transactions for block 57833080 (this will cache the result)
filtered_transactions = cache.get_filtered_transactions(57833080)
filtered_transactions = cache.get_filtered_transactions(57833084)
# Retrieve filtered transactions again for the same block (this will use the cache)
#filtered_transactions_again = cache.get_filtered_transactions(57833080)

# Print the filtered transactions
#print(filtered_transactions)

In [106]:
cache.print_cached_transactions()

Block Number: 57833080
Transactions:
{'log': [{'address': '0fa695d6b065707cb4e0ef73b751c93347682bf2', 'topics': ['4a504a94899432a9846e1aa406dceb1bcfd538bb839071d49d1e5e23f5be30ef', '000000000000000000000000a89ae0805bbc6b0d7912d7103f216f3f2ff04f97', '0000000000000000000000000000000000000000000000000000000000000419']}, {'address': 'a614f803b6fd780986a42c78ec9c7f77e6ded13c', 'topics': ['42e160154868087d6bfdc0ca23d96a1c1cfa32f1b72ba9ba27b69b98a0d819dc', '000000000000000000000000c67d47af3dc61a89364f47bd00183c0700cf4ed3']}, {'address': '0fa695d6b065707cb4e0ef73b751c93347682bf2', 'topics': ['33e13ecb54c3076d8e8bb8c2881800a4d972b792045ffae98fdf46df365fed75', '0000000000000000000000000000000000000000000000000000000000000419']}], 'blockNumber': 57833080, 'contractResult': [''], 'blockTimeStamp': 1704132882000, 'receipt': {'result': 'SUCCESS', 'energy_penalty_total': 26102, 'energy_usage': 95895, 'energy_usage_total': 95895, 'net_usage': 313}, 'id': 'b3b38075f9987af64f56c7059f9adb1dd4233de2af36d2

In [115]:
block_last=57833080
offset = 100

cache = BlockTransactionCache()

for block_number in tqdm(range(block_last, block_last+offset)):
    cache.get_filtered_transactions(block_number)

100%|██████████| 100/100 [00:23<00:00,  4.19it/s]


In [75]:
#cache.print_cached_transactions()

In [121]:
cache.aggregate_cache_to_csv()

CSV file 'aggregated_transactions.csv' has been created successfully.


In [122]:
cache.clear_cache() 

Cache has been cleared.


In [123]:
cache.print_cached_transactions()

No transactions in cache.


In [124]:
# Конфігурація для багатопотокової обробки
batch_size = 100  # наприклад, розмір пачки
offset = 0  # початковий зсув
max_workers = 3  # кількість потоків

In [125]:
# Get the number of logical CPU cores
num_cores = os.cpu_count()

# Get available memory in MB
available_memory = psutil.virtual_memory().available / (1024 * 1024)

# Estimate the memory usage per thread (in MB)
memory_per_thread = 100  # Adjust based on your task's requirements
# Calculate max workers based on memory
max_workers_memory = int(available_memory // memory_per_thread)

# Choose the smaller value between CPU-bound and memory-bound limits
max_workers = min(num_cores * 2, max_workers_memory)  # Assuming I/O-bound tasks
print(f"Optimal max_workers: {max_workers}")

Optimal max_workers: 24


In [ ]:
cache = BlockTransactionCache()

batch_size = 100
block_last = 57833080

while block_last < 57834080: #69200000:
    
    cache.get_filtered_transactions(block_last)
    process_batch(block_last, batch_size)
    block_last =  block_last + batch_size
    delete_files_with_prefix("logs")

In [141]:
def process_batch(cache, block_last, batch_size):
    
        # Process blocks in parallel using ThreadPoolExecutor
        block_last = block_last  # Adjust to your starting block
        block_numbers = range(block_last, block_last + batch_size)  # Adjust the range of blocks you want to process

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_block = {executor.submit(cache.get_filtered_transactions, block_number): block_number for block_number in block_numbers}

            # Track progress with tqdm for parallel tasks
            for future in tqdm(as_completed(future_to_block), total=len(block_numbers), desc="Processing blocks", unit="block"):
                block_number = future_to_block[future]
                try:
                    future.result()  # Wait for the block processing to finish
                except Exception as e:
                    continue
                    #print(f"Block {block_number} generated an error: {e}")
                    
        #aggregate_and_cleanup_csvs(f"aggregated_transactions-{block_last}.csv", prefix="logs_block")
        cache.aggregate_cache_to_csv(filename=f"aggregated_transactions-{block_last}.csv")
        cache.clear_cache() 

In [147]:
cache = BlockTransactionCache()

batch_size = 10000
block_last = 58000000#57833080

while block_last < 58500000:#58000000: #69200000:

    process_batch(cache, block_last, batch_size)
    block_last =  block_last + batch_size

Processing blocks: 100%|██████████| 10000/10000 [01:14<00:00, 133.41block/s]


No transactions to write to CSV.
Cache has been cleared.


Processing blocks: 100%|██████████| 10000/10000 [01:12<00:00, 138.37block/s]


No transactions to write to CSV.
Cache has been cleared.


Processing blocks: 100%|██████████| 10000/10000 [01:22<00:00, 121.65block/s]


...

No transactions to write to CSV.
Cache has been cleared.


Processing blocks: 100%|██████████| 10000/10000 [01:21<00:00, 121.99block/s]


No transactions to write to CSV.
Cache has been cleared.


Processing blocks: 100%|██████████| 10000/10000 [01:18<00:00, 127.19block/s]


No transactions to write to CSV.
Cache has been cleared.
